In [1]:
# coding: utf-8
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import pandas as pd

import h1st.core as h1
h1.init()

from examples.RayTune.sklearn_smv_classifier import SklearnSVMClassifier
from examples.RayTune.tensorflow_mlp_classifier import TensorflowMLPClassifier
from examples.RayTune import config
from examples.RayTune.utils import prepare_train_test_data

## Tune's Functional API

In [ ]:
import ray
from ray import tune

In [ ]:
ray.init(ignore_reinit_error=True)

In [ ]:
config = {
    "lr": tune.grid_search([0.005]),
    "units": tune.grid_search([16, 32, 64, 128]),
    "n_layer": tune.grid_search([3, 5, 10])
}

In [ ]:
DATA_ROOT = './data/credit_card'

def train_model_with_tune(config):
    from ray.tune import report
    import os
    from filelock import FileLock    
    h1_tf_mlp = TensorflowMLPClassifier(
        units=config['units'], lr=config['lr'], n_layer=config['n_layer'])
    
    lock_file = f'{DATA_ROOT}/data.lock'
    if not os.path.exists(DATA_ROOT):
        os.makedirs(DATA_ROOT)
    with FileLock(os.path.expanduser(lock_file)):
        data = h1_tf_mlp.load_data()
    prepared_data = h1_tf_mlp.prep(data) 
 
    h1_tf_mlp.train(prepared_data)
    h1_tf_mlp.evaluate(prepared_data)
    for _ in range(5):
        report(iterations=100, mean_accuracy=h1_tf_mlp.metrics['accuracy'])    

#     h1_tf_mlp.persist('my_tf_mlp')
#     report(mean_accuracy=h1_tf_mlp.metrics['accuracy'])

#     train_loader, test_loader = get_data_loaders()
#     model = ConvNet()
#     optimizer = optim.SGD(model.parameters(), lr=config["lr"], momentum=config['momentum'])
#     for i in range(10):
#         train(model, optimizer, train_loader)
#         acc = test(model, test_loader)
        

In [ ]:
analysis_func = tune.run(train_model_with_tune, config=config, stop={"training_iteration": 10})

In [ ]:
print("Best config: ", analysis_func.get_best_config(metric="mean_accuracy"))

In [ ]:
analysis_func.dataframe().sort_values('mean_accuracy', ascending=False).head()

In [ ]:
analysis_func.dataframe()[['mean_accuracy', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False)

In [ ]:
stats = analysis_func.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

In [ ]:
ray.shutdown()

In [ ]:
# h1_tf_mlp = TensorflowMLPClassifier(units=32, lr=0.01)
# data = h1_tf_mlp.load_data()
# prepared_data = h1_tf_mlp.prep(data)
# h1_tf_mlp.train(prepared_data)
# h1_tf_mlp.evaluate(prepared_data)
# h1_tf_mlp.metrics
# h1_tf_mlp.persist('my_tf_mlp')

## Tune's Class API

In [ ]:
ray.init(ignore_reinit_error=True)

In [ ]:
class TrainH1stModel(tune.Trainable):
    def _setup(self, config):
        from ray.tune import report
        import os
        from filelock import FileLock            
        self.hyper_parameters = {
            'units': config['units'], 'n_layer': config['n_layer']}
        self.h1_tf_mlp = TensorflowMLPClassifier(lr=0.01, config['units'],  config['n_layer'])
        DATA_ROOT = './data/credit_card'
        lock_file = f'{DATA_ROOT}/data.lock'
        if not os.path.exists(DATA_ROOT):
            os.makedirs(DATA_ROOT)
        with FileLock(os.path.expanduser(lock_file)):
            data = self.h1_tf_mlp.load_data()
        self.prepared_data = self.h1_tf_mlp.prep(data)         
        
    def _train(self):
        self.h1_tf_mlp.train(self.prepared_data)
        self.h1_tf_mlp.evaluate(self.prepared_data)
        acc = self.h1_tf_mlp.metrics['accuracy']
        return {"mean_accuracy": acc}

In [ ]:
config = {
    "lr": tune.grid_search([0.005]),
    "units": tune.grid_search([16, 32, 64, 128]),
    "n_layer": tune.grid_search([3, 5, 10])
#     "n_layer": tune.uniform(1, 10)
}
 
analysis = tune.run(
    TrainH1stModel, 
    config=config, 
    verbose=2,
    stop={"training_iteration": 3}
)

## Population Base Training

In [ ]:
# common

import json
import os
import datetime

class TrainH1stModel(tune.Trainable):
    def _setup(self, config):
        from filelock import FileLock 
        self.config = config
        self.timestep = 0
        self.h1_tf_mlp = TensorflowMLPClassifier(
            units=int(config['units']), 
            epochs=int(config['epochs']), 
            lr=config['lr'], 
            n_layer=int(config['n_layer']))
        DATA_ROOT = './data/credit_card'
        lock_file = f'{DATA_ROOT}/data.lock'
        if not os.path.exists(DATA_ROOT):
            os.makedirs(DATA_ROOT)
        with FileLock(os.path.expanduser(lock_file)):
            data = self.h1_tf_mlp.load_data()
        self.prepared_data = self.h1_tf_mlp.prep(data)         
        
    def _train(self):
        self.timestep += 1
        self.h1_tf_mlp.train(self.prepared_data)
        self.h1_tf_mlp.evaluate(self.prepared_data)

        # tf_mlp_{timestep}_{lr}_{layer}_{unit}_{epochs}_{ts}
        version_name = "tf_mlp_" + (
            "_".join(f"{k}:{v}" for k, v in self.config.items())
        ) + "_" + str(int(datetime.datetime.utcnow().timestamp()))

        version = self.h1_tf_mlp.persist(version_name)
        acc = self.h1_tf_mlp.metrics['accuracy']
        return {"mean_accuracy": acc, 'model_version': version}
    
    def _save(self, checkpoint_dir):
        import json
        path = os.path.join(checkpoint_dir, "checkpoint")
        with open(path, "w") as f:
            f.write(json.dumps({"timestep": self.timestep}))
        return path

    def _restore(self, checkpoint_path):
        import json
        with open(checkpoint_path) as f:
            self.timestep = json.loads(f.read())["timestep"]

In [ ]:
# XXX 2

from ray.tune.schedulers import PopulationBasedTraining

experiment_metrics = dict(metric="mean_accuracy", mode="max")

pbt_scheduler = PopulationBasedTraining(
        time_attr='training_iteration',
        perturbation_interval=2,  # Every N time_attr units, "perturb" the parameters.
        hyperparam_mutations={           
            "lr": [0.005],
            "epochs": [2, 10, 30, 100],
            "units": [4, 8, 16, 32, 64],
            "n_layer": [2, 4, 6]            
        },
        **experiment_metrics)

# Note: This appears to be needed to avoid a "key error", but in fact these values won't change
# in the analysis.dataframe() object, even though they will be tuned by the PBT scheduler.
# So when you look at the analysis.dataframe(), look at the `experiment_tag` to see the actual values!
config = {
    "lr": 0.005,            # Use the lowest values from the previous definition
    "units": 16,
    "n_layer": 4,
    "epochs": 2
}

In [ ]:
# XXX 3

analysis = tune.run(TrainH1stModel, 
    scheduler=pbt_scheduler, 
    config=config,
    stop={"training_iteration": 4},
#     stop={"mean_accuracy": 0.82, "training_iteration": 200},                    
    num_samples=40,
    verbose=1
)

stats = analysis.stats()
secs = stats["timestamp"] - stats["start_time"]
print(f'{secs:7.2f} seconds, {secs/60.0:7.2f} minutes')

In [ ]:
analysis.dataframe()[['trial_id', 'mean_accuracy', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False)

## BO Hyperband (BOHB)

In [ ]:
# import json
# import os

# import numpy as np

# import ray
# from ray import tune
# from ray.tune import Trainable
# from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
# from ray.tune.suggest.bohb import TuneBOHB


# class MyTrainableClass(Trainable):
#     """Example agent whose learning curve is a random sigmoid.

#     The dummy hyperparameters "width" and "height" determine the slope and
#     maximum reward value reached.
#     """

#     def setup(self, config):
#         self.timestep = 0

#     def step(self):
#         self.timestep += 1
#         v = np.tanh(float(self.timestep) / self.config.get("width", 1))
#         v *= self.config.get("height", 1)

#         # Here we use `episode_reward_mean`, but you can also report other
#         # objectives such as loss or accuracy.
#         return {"episode_reward_mean": v}

#     def save_checkpoint(self, checkpoint_dir):
#         path = os.path.join(checkpoint_dir, "checkpoint")
#         with open(path, "w") as f:
#             f.write(json.dumps({"timestep": self.timestep}))
#         return path

#     def load_checkpoint(self, checkpoint_path):
#         with open(checkpoint_path) as f:
#             self.timestep = json.loads(f.read())["timestep"]


# if __name__ == "__main__":
#     import ConfigSpace as CS  # noqa: F401
#     ray.init(num_cpus=8)

#     config = {
#         "iterations": 100,
#         "width": tune.uniform(0, 20),
#         "height": tune.uniform(-100, 100),
#         "activation": tune.choice(["relu", "tanh"])
#     }

#     # Optional: Pass the parameter space yourself
#     # config_space = CS.ConfigurationSpace()
#     # config_space.add_hyperparameter(
#     #     CS.UniformFloatHyperparameter("width", lower=0, upper=20))
#     # config_space.add_hyperparameter(
#     #     CS.UniformFloatHyperparameter("height", lower=-100, upper=100))
#     # config_space.add_hyperparameter(
#     #     CS.CategoricalHyperparameter(
#     #         "activation", choices=["relu", "tanh"]))

#     experiment_metrics = dict(metric="episode_reward_mean", mode="max")

#     bohb_hyperband = HyperBandForBOHB(
#         time_attr="training_iteration",
#         max_t=100,
#         reduction_factor=4,
#         **experiment_metrics)

#     bohb_search = TuneBOHB(
#         # space=config_space,  # If you want to set the space manually
#         max_concurrent=4,
#         **experiment_metrics)

#     tune.run(
#         MyTrainableClass,
#         name="bohb_test",
#         config=config,
#         scheduler=bohb_hyperband,
#         search_alg=bohb_search,
#         num_samples=10,
#         stop={"training_iteration": 100})

In [ ]:
import ConfigSpace as CS
from ray.tune.schedulers.hb_bohb import HyperBandForBOHB
from ray.tune.suggest.bohb import TuneBOHB

In [ ]:
config_space = CS.ConfigurationSpace()

# There are also UniformIntegerHyperparameter and UniformFloatHyperparameter
# objects for defining integer and float ranges, respectively. For example:
# config_space.add_hyperparameter(
#     CS.UniformIntegerHyperparameter('foo', lower=0, upper=100))

# config_space.add_hyperparameter(
#     CS.UniformFloatHyperparameter('lr', lower=0.001, upper=0.1))
# config_space.add_hyperparameter(
#     CS.UniformIntegerHyperparameter('units', lower=1, upper=64))
# config_space.add_hyperparameter(
#     CS.UniformIntegerHyperparameter('n_layer', lower=1, upper=8))

config_space.add_hyperparameter(
    CS.CategoricalHyperparameter('lr', choices=[0.005]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter('units', choices=[4, 8, 16, 32]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter('n_layer', choices=[2, 4, 6, 8]))
config_space.add_hyperparameter(
    CS.CategoricalHyperparameter('epochs', choices=[2, 10, 30, 100]))

In [ ]:
experiment_metrics = dict(metric="mean_accuracy", mode="max")
search_algorithm = TuneBOHB(config_space, max_concurrent=4, **experiment_metrics)
# search_algorithm = TuneBOHB(config_space, max_concurrent=4)

scheduler = HyperBandForBOHB(
    time_attr='training_iteration',
    reduction_factor=4,)
#     **experiment_metrics)

In [ ]:
analysis = tune.run(
    TrainH1stModel, 
    scheduler=scheduler, 
    search_alg=search_algorithm, 
    metric="mean_accuracy",
    mode="max",         
    num_samples=40,                           # Force it try all 12 combinations
    stop={"training_iteration": 4},
    verbose=1
)

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/epochs','config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False)

## Bayesian Optimization (BO)

In [ ]:
"""This test checks that BayesOpt is functional.

It also checks that it is usable with a separate scheduler.
"""
import time


import ray
from ray import tune
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.suggest.bayesopt import BayesOptSearch


# def evaluation_fn(step, width, height):
#     return (0.1 + width * step / 100)**(-1) + height * 0.1


# def easy_objective(config):
#     # Hyperparameters
#     width, height = config["width"], config["height"]

#     for step in range(config["steps"]):
#         # Iterative training function - can be any arbitrary training procedure
#         intermediate_score = evaluation_fn(step, width, height)
#         # Feed the score back back to Tune.
#         tune.report(iterations=step, mean_loss=intermediate_score)
#         time.sleep(0.1)

space = {
    "lr": tune.uniform(0.005, 0.05),
    "units": tune.quniform(1, 64, 1),
#     "units": tune.randint(1, 64),
#     "n_layer": tune.randint(1, 8)
    "n_layer": tune.quniform(1, 8, 1),    
    "epochs": tune.quniform(1, 50, 1),
}

algo = BayesOptSearch(   
    utility_kwargs={
        "kind": "ucb",
        "kappa": 2.5,
        "xi": 0.0
    }
)

algo = ConcurrencyLimiter(algo, max_concurrent=4)
# scheduler = AsyncHyperBandScheduler()
analysis= tune.run(
    TrainH1stModel,
    config=space,
    metric="mean_accuracy",
    mode="max",     
#     name="my_exp",
    search_alg=algo,
#     scheduler=scheduler,
    stop={"training_iteration": 4},
    verbose=1,
    num_samples=20,
)


In [ ]:
analysis.dataframe()[0:10]

In [ ]:
analysis.dataframe()[['mean_accuracy', 'config/lr', 'config/n_layer', 'config/units']].sort_values('mean_accuracy', ascending=False)

## OptunaSearch

## HyperOptSearch